In [12]:
import numpy as np
import pandas as pd
import os
import re
import shutil

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical, image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold

import cv2
from PIL import Image
matplotlib.use('Agg')   # type: ignore
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [13]:
# Define dataset directory
images_dir = "images"
# "C:/Users/theof/OneDrive/Documents/Github/genome_color_unpickler/train-raw/arrays"

# Training parameters
img_size = (32, 32)
batch_size = 32
epochs = 15

# Dictionary to store accuracies
max_accuracies = {}

In [14]:
# Function to build a simple CNN model
def cnn_model(input_shape, num_classes):
	model = Sequential([
		Conv2D(32, (5,5), activation='relu', input_shape=input_shape),
		MaxPooling2D(2,2),
		Conv2D(64, (5,5), activation='relu'),
		MaxPooling2D(2,2),
		Flatten(),
		Dense(128, activation='relu'),
		Dense(num_classes, activation='softmax')  # Adjust output neurons based on classes
	])
	model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [15]:
# Iterate through resolution folders
for resolution_folder in sorted(os.listdir(images_dir), key=lambda x: int(re.search(r'\d+', x).group())):
	resolution_path = os.path.join(images_dir, resolution_folder)

	if os.path.isdir(resolution_path):
		# Iterate through method folders inside resolution
		for method_folder in os.listdir(resolution_path):
			method_path = os.path.join(resolution_path, method_folder)

			if os.path.isdir(method_path):
				print(f"\nTraining {resolution_folder}/{method_folder}...")

				# Data Preprocessing
				datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

				# Use subset='training' and 'validation' for split
				train_data = datagen.flow_from_directory(
					method_path,
					target_size=img_size,
					batch_size=batch_size,
					class_mode='categorical',
					subset='training'
				)
				val_data = datagen.flow_from_directory(
					method_path,
					target_size=img_size,
					batch_size=batch_size,
					class_mode='categorical',
					subset='validation',
					shuffle=False
				)

				num_classes = train_data.num_classes
				input_shape = (img_size[0], img_size[1], 3)

				# Initialize and train the model
				model = cnn_model(input_shape, num_classes)
				history = model.fit(
					train_data,
					validation_data=val_data,
					epochs=epochs,
					verbose=1
				)

				# Save the model
				model_path = os.path.join("models/standard", resolution_folder, method_folder)
				os.makedirs(model_path, exist_ok=True)
				model_name = f"{resolution_folder}_{method_folder}_mask_5_std.h5"
				model.save(os.path.join(model_path, model_name))
				print(f"Model saved to {model_path}")

				# Store final validation accuracy
				final_val_acc = max(history.history['val_accuracy'])
				key = f"{resolution_folder} - {method_folder}"
				max_accuracies[key] = final_val_acc
				print(f"Validation accuracy for {key}: {final_val_acc:.4f}")



Training 4px/Chargaff-Composante-Diversite...
Found 5419 images belonging to 5 classes.
Found 1352 images belonging to 5 classes.
Epoch 1/15
170/170 [==============================] - 5s 26ms/step - loss: 0.9656 - accuracy: 0.6545 - val_loss: 0.9599 - val_accuracy: 0.6516
Epoch 2/15
170/170 [==============================] - 3s 19ms/step - loss: 0.9076 - accuracy: 0.6675 - val_loss: 0.9369 - val_accuracy: 0.6280
Epoch 3/15
170/170 [==============================] - 3s 18ms/step - loss: 0.8899 - accuracy: 0.6822 - val_loss: 0.9463 - val_accuracy: 0.6746
Epoch 4/15
170/170 [==============================] - 3s 18ms/step - loss: 0.8822 - accuracy: 0.6902 - val_loss: 0.9814 - val_accuracy: 0.6531
Epoch 5/15
170/170 [==============================] - 3s 19ms/step - loss: 0.8691 - accuracy: 0.6889 - val_loss: 0.8956 - val_accuracy: 0.6716
Epoch 6/15
170/170 [==============================] - 3s 19ms/step - loss: 0.8639 - accuracy: 0.6903 - val_loss: 0.8720 - val_accuracy: 0.6938
Epoch 7/15


c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 1352 images belonging to 5 classes.
Epoch 1/15
170/170 [==============================] - 5s 26ms/step - loss: 0.9973 - accuracy: 0.6468 - val_loss: 0.9626 - val_accuracy: 0.6516
Epoch 2/15
170/170 [==============================] - 3s 18ms/step - loss: 0.9712 - accuracy: 0.6512 - val_loss: 0.9469 - val_accuracy: 0.6516
Epoch 3/15
170/170 [==============================] - 3s 18ms/step - loss: 0.9078 - accuracy: 0.6828 - val_loss: 0.8151 - val_accuracy: 0.7448
Epoch 4/15
170/170 [==============================] - 3s 18ms/step - loss: 0.8836 - accuracy: 0.7057 - val_loss: 0.8232 - val_accuracy: 0.7389
Epoch 5/15
170/170 [==============================] - 3s 19ms/step - loss: 0.8583 - accuracy: 0.7247 - val_loss: 0.7937 - val_accuracy: 0.7485
Epoch 6/15
170/170 [==============================] - 3s 19ms/step - loss: 0.8533 - accuracy: 0.7226 - val_loss: 0.8015 - val_accuracy: 0.7567
Epoch 7/15
170/170 [==============================] - 3s 18ms/step - loss: 0.8448 - accuracy: 0.7346

In [16]:
# Create positions for bars
num_bars = len(max_accuracies)
x_positions = np.arange(num_bars)

# Add extra space every 10 bars
for i in range(10, num_bars, 10):  
	x_positions[i:] += 1  # Shift everything after every 10th bar

plt.figure(figsize=(15, 10))
plt.bar(x_positions, max_accuracies.values(), color='blue', width=0.5)
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Custom x-ticks for readability
plt.xticks(x_positions, max_accuracies.keys(), rotation=90)

plt.yticks(np.arange(0, 1.1, 0.2))

plt.ylabel("Validation Accuracy")
plt.title("CNN Validation Results")
plt.tight_layout()
plt.show()

# plt.savefig("../../imgs/graphs/kfold/cnn_validation_accuracy_bar_mask_5-kfold_aug2.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_44516\22334614.py:21: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [17]:
# Extract and sort resolutions numerically
resolutions = sorted(set(k.split(" - ")[0] for k in max_accuracies.keys()), key=lambda x: int(x.replace("px", "")))
methods = sorted(set(k.split(" - ")[1] for k in max_accuracies.keys()))

# Organize data for plotting
data = {method: [max_accuracies.get(f"{res} - {method}", None) for res in resolutions] for method in methods}

# Define a color map for better distinction
colors = plt.cm.get_cmap("tab10", len(methods))  # Use "tab10" color map with enough colors

# Plot the lines
plt.figure(figsize=(16, 10))
for i, (method, accuracies) in enumerate(data.items()):
	plt.plot(resolutions, accuracies, marker="o", linestyle="-", linewidth=2, markersize=8, label=method, color=colors(i))

# Labels and title
plt.xlabel("Resolution", fontsize=12)
plt.ylabel("Max Accuracy", fontsize=12)
plt.title("CNN Validation Accuracy by Resolution and Method", fontsize=14)

# Move legend outside the plot for better clarity
plt.legend(title="Method", bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=10)

# Add a grid with transparency
plt.grid(True, linestyle="--", alpha=0.6)

# Improve layout to fit legend properly
plt.tight_layout()

# Show plot
plt.show()

plt.savefig("../../imgs/graphs/standard/cnn_validation_accuracy_mask_5_std.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_44516\1426259004.py:9: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = plt.cm.get_cmap("tab10", len(methods))  # Use "tab10" color map with enough colors
C:\Users\theof\AppData\Local\Temp\ipykernel_44516\1426259004.py:31: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [18]:
# Extract and sort resolutions numerically
resolutions = sorted(set(k.split(" - ")[0] for k in max_accuracies.keys()), key=lambda x: int(x.replace("px", "")))

# Extract unique methods and split into two groups of 5
methods = sorted(set(k.split(" - ")[1] for k in max_accuracies.keys()))
methods_group1 = methods[:5]
methods_group2 = methods[5:]

# Organize data for plotting
data1 = {method: [max_accuracies.get(f"{res} - {method}", None) for res in resolutions] for method in methods_group1}
data2 = {method: [max_accuracies.get(f"{res} - {method}", None) for res in resolutions] for method in methods_group2}

# Create subplots
fig, axs = plt.subplots(2, 1, figsize=(10, 10), sharex=True, sharey=True)

colors = plt.cm.get_cmap("tab10", len(methods_group1))  # Use "tab10" color map with enough colors

# Plot first group
for method, accuracies in data1.items():
	axs[0].plot(resolutions, accuracies, marker="o", label=method, color=colors(methods_group1.index(method)))
axs[0].set_title("CNN Validation Accuracy (Group 1)")
axs[0].set_ylabel("Max Accuracy")
axs[0].legend(title="Method")
axs[0].grid(True, linestyle="--", alpha=0.6)

# Plot second group
for method, accuracies in data2.items():
	axs[1].plot(resolutions, accuracies, marker="o", label=method, color=colors(methods_group2.index(method)))
axs[1].set_title("CNN Validation Accuracy (Group 2)")
axs[1].set_xlabel("Resolution")
axs[1].set_ylabel("Max Accuracy")
axs[1].legend(title="Method")
axs[1].grid(True, linestyle="--", alpha=0.6)

# Adjust layout
plt.tight_layout()
plt.show()

plt.savefig("../../imgs/graphs/standard/cnn_validation_accuracy_groups_mask_5_std.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_44516\361041927.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = plt.cm.get_cmap("tab10", len(methods_group1))  # Use "tab10" color map with enough colors
C:\Users\theof\AppData\Local\Temp\ipykernel_44516\361041927.py:37: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [19]:
# confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from keras.models import load_model

# Load the model
model_path = "models/standard/100px/Chargaff-Composante-Diversite/100px_Chargaff-Composante-Diversite_mask_5_std.h5"
model = load_model(model_path)
model.summary()

# Load the test data
test_data_dir = 'images/100px/Chargaff-Composante-Diversite'
# 'C:/Users/theof/OneDrive/Documents/Github/genome_color_unpickler/test-raw/arrays/100px/Chargaff-Composante-Diversite'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
	test_data_dir,
	target_size=img_size,
	batch_size=batch_size,
	class_mode='categorical',
	shuffle=False
)
# Get the true labels
true_labels = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Get the predicted labels
predictions = model.predict(test_generator, steps=len(test_generator), verbose=1)
predicted_labels = np.argmax(predictions, axis=1)
# Generate the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Confusion Matrix at 100px Resolution for Chargaff-Composante-Diversite Method')
plt.tight_layout()
plt.show()

plt.savefig("../../imgs/graphs/standard/cnn_confusion_matrix_100px_mask_5_std.png")

Model: "sequential_90"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_180 (Conv2D)         (None, 28, 28, 32)        2432      
                                                                 
 max_pooling2d_180 (MaxPool  (None, 14, 14, 32)        0         
 ing2D)                                                          
                                                                 
 conv2d_181 (Conv2D)         (None, 10, 10, 64)        51264     
                                                                 
 max_pooling2d_181 (MaxPool  (None, 5, 5, 64)          0         
 ing2D)                                                          
                                                                 
 flatten_90 (Flatten)        (None, 1600)              0         
                                                                 
 dense_180 (Dense)           (None, 128)             

C:\Users\theof\AppData\Local\Temp\ipykernel_44516\3230203911.py:40: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [39]:
# Define dataset directory
mosaics_dir = "mosaics"
# "C:/Users/theof/OneDrive/Documents/Github/genome_color_unpickler/train-raw/mosaics"

# Training parameters
img_size = (20, 50)  # Resize images to 64x64
batch_size = 32
epochs = 15
 
# Dictionary to store max accuracies
max_accuracies_mos = {}

In [40]:
# Mosaic version (without cross-validation)
# Iterate through resolution folders (e.g., 4px, 16px)
for resolution_folder in sorted(os.listdir(mosaics_dir), key=lambda x: int(re.search(r'\d+', x).group())):
	resolution_path = os.path.join(mosaics_dir, resolution_folder)

	if os.path.isdir(resolution_path):
		print(f"\nTraining {resolution_folder}...")

		# Data Preprocessing
		datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

		# Create training and validation generators
		train_data = datagen.flow_from_directory(
			resolution_path,
			target_size=img_size,
			batch_size=batch_size,
			class_mode='categorical',
			subset='training'
		)
		val_data = datagen.flow_from_directory(
			resolution_path,
			target_size=img_size,
			batch_size=batch_size,
			class_mode='categorical',
			subset='validation',
			shuffle=False
		)

		num_classes = train_data.num_classes
		input_shape = (img_size[0], img_size[1], 3)

		# Initialize and train model
		model = cnn_model(input_shape, num_classes)
		history = model.fit(
			train_data,
			validation_data=val_data,
			epochs=epochs,
			verbose=1
		)

		# Save the model
		model_path = os.path.join("models/standard-mosaic", resolution_folder)
		os.makedirs(model_path, exist_ok=True)
		model_name = f"{resolution_folder}_mosaic_mask_5_std.h5"
		model.save(os.path.join(model_path, model_name))
		print(f"Model saved to {model_path}")

		# Store final validation accuracy
		final_val_acc = max(history.history['val_accuracy'])
		max_accuracies_mos[resolution_folder] = final_val_acc
		print(f"Validation accuracy for {resolution_folder}: {final_val_acc:.4f}")


Training 4px...
Found 5419 images belonging to 5 classes.
Found 1352 images belonging to 5 classes.
Epoch 1/15
170/170 [==============================] - 4s 18ms/step - loss: 0.9646 - accuracy: 0.6523 - val_loss: 0.9638 - val_accuracy: 0.6494
Epoch 2/15
170/170 [==============================] - 3s 17ms/step - loss: 0.9226 - accuracy: 0.6658 - val_loss: 0.9175 - val_accuracy: 0.6738
Epoch 3/15
170/170 [==============================] - 3s 16ms/step - loss: 0.8905 - accuracy: 0.6820 - val_loss: 0.9036 - val_accuracy: 0.6768
Epoch 4/15
170/170 [==============================] - 3s 16ms/step - loss: 0.8807 - accuracy: 0.6861 - val_loss: 0.9346 - val_accuracy: 0.6679
Epoch 5/15
170/170 [==============================] - 3s 16ms/step - loss: 0.8788 - accuracy: 0.6837 - val_loss: 0.9011 - val_accuracy: 0.6709
Epoch 6/15
170/170 [==============================] - 3s 17ms/step - loss: 0.8628 - accuracy: 0.6889 - val_loss: 0.8630 - val_accuracy: 0.6960
Epoch 7/15
170/170 [=====================

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 1352 images belonging to 5 classes.
Epoch 1/15
170/170 [==============================] - 4s 18ms/step - loss: 0.8552 - accuracy: 0.7151 - val_loss: 0.7613 - val_accuracy: 0.7419
Epoch 2/15
170/170 [==============================] - 3s 18ms/step - loss: 0.6059 - accuracy: 0.8057 - val_loss: 0.7691 - val_accuracy: 0.7493
Epoch 3/15
170/170 [==============================] - 3s 18ms/step - loss: 0.4636 - accuracy: 0.8527 - val_loss: 0.7395 - val_accuracy: 0.7633
Epoch 4/15
170/170 [==============================] - 3s 16ms/step - loss: 0.3947 - accuracy: 0.8727 - val_loss: 0.3828 - val_accuracy: 0.8876
Epoch 5/15
170/170 [==============================] - 3s 17ms/step - loss: 0.3615 - accuracy: 0.8825 - val_loss: 0.4040 - val_accuracy: 0.8713
Epoch 6/15
170/170 [==============================] - 3s 16ms/step - loss: 0.3301 - accuracy: 0.8957 - val_loss: 0.4227 - val_accuracy: 0.8683
Epoch 7/15
170/170 [==============================] - 3s 16ms/step - loss: 0.3066 - accuracy: 0.9051

In [41]:
# Create positions for bars
num_bars = len(max_accuracies_mos)
x_positions = np.arange(num_bars)

plt.figure(figsize=(15, 10))
plt.bar(x_positions, max_accuracies_mos.values(), color='blue', width=0.5)
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Custom x-ticks for readability
plt.xticks(x_positions, max_accuracies_mos.keys(), rotation=90)

plt.yticks(np.arange(0, 1.1, 0.2))

plt.ylabel("Max Accuracy")
plt.title("CNN Validation Results")
plt.tight_layout()
plt.show()

# plt.savefig("../../imgs/graphs/standard_mosaic/cnn_validation_accuracy_mosaics_mask_5_std.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_44516\2427635834.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [42]:
# Extract and sort resolutions numerically
resolutions = sorted(max_accuracies_mos.keys(), key=lambda x: int(x.replace("px", "")))

# Get max accuracy values in the correct order
accuracies = [max_accuracies_mos[res] for res in resolutions]

# Plot the accuracy per resolution
plt.figure(figsize=(12, 6))
plt.plot(resolutions, accuracies, marker="o", linestyle="-", linewidth=2, markersize=8, color="blue", label="Accuracy")

# Labels and title
plt.xlabel("Resolution", fontsize=12)
plt.ylabel("Max Accuracy", fontsize=12)
plt.title("CNN Validation Accuracy by Resolution", fontsize=14)

# Add data points on the plot
for i, acc in enumerate(accuracies):
    plt.text(resolutions[i], acc, f"{acc:.2f}", fontsize=10, ha="right")

# Grid and legend
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend(loc="lower right")

# Save and show plot
plt.tight_layout()
plt.show()
plt.savefig("../../imgs/graphs/standard-mosaic/cnn_validation_accuracy_mosaics_line_mask_5_std.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_44516\2161438940.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [43]:
# confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from keras.models import load_model

# Load the model for the new version
new_model_path = "models/standard-mosaic/100px/100px_mosaic_mask_5_aug.h5"
new_model = load_model(new_model_path)
new_model.summary()

# Load the test data for the new version
new_test_data_dir = 'mosaics/100px'
# 'C:/Users/theof/OneDrive/Documents/Github/genome_color_unpickler/test-raw/mosaics/100px'

new_test_datagen = ImageDataGenerator(rescale=1./255)
new_test_generator = new_test_datagen.flow_from_directory(
	new_test_data_dir,
	target_size=(img_size[0], img_size[1]),
	batch_size=batch_size,
	class_mode='categorical',
	shuffle=False
)

# Get the true labels for the new version
new_true_labels = new_test_generator.classes
new_class_labels = list(new_test_generator.class_indices.keys())

# Get the predicted labels for the new version
new_predictions = new_model.predict(new_test_generator, steps=len(new_test_generator), verbose=1)
new_predicted_labels = np.argmax(new_predictions, axis=1)

# Generate the confusion matrix for the new version
new_cm = confusion_matrix(new_true_labels, new_predicted_labels)

# Plot the confusion matrix for the new version
plt.figure(figsize=(10, 8))
sns.heatmap(new_cm, annot=True, fmt='d', cmap='Blues', xticklabels=new_class_labels, yticklabels=new_class_labels)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Confusion Matrix for Mosaic images at 100px')
plt.tight_layout()
plt.show()

plt.savefig("../../imgs/graphs/standard-mosaic/cnn_confusion_matrix_mosaics_100px_mask_5_std.png")

OSError: No file or directory found at models/standard-mosaic/100px/100px_mosaic_mask_5_aug.h5